In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from PIL.ExifTags import TAGS
from datetime import datetime
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import shutil
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoImageProcessor, AutoModel, AutoFeatureExtractor
import tqdm
from pathlib import Path

# 사람 감지를 위한 모델 추가
try:
    from transformers import DetrImageProcessor, DetrForObjectDetection
except ImportError:
    print("DETR 모델을 위한 transformers 라이브러리가 필요합니다.")

# 고정된 입력 및 출력 디렉토리 설정
IMAGE_DIR = "./1차"
OUTPUT_DIR = IMAGE_DIR + "_claude"

class ImageDataset(Dataset):
    """이미지 모델용 데이터셋 클래스"""
    def __init__(self, image_paths, transform=None, remove_people=False, people_detector=None):
        self.image_paths = image_paths
        self.transform = transform
        self.remove_people = remove_people
        self.people_detector = people_detector
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        try:
            # PIL로 이미지 읽기
            image = Image.open(image_path).convert('RGB')
            
            # 사람 제거 옵션이 활성화된 경우
            if self.remove_people and self.people_detector is not None:
                image = self.mask_people(image)
            
            if self.transform:
                image = self.transform(image)
            
            return {
                'path': image_path,
                'image': image
            }
        except Exception as e:
            print(f"이미지 {image_path} 로딩 중 오류: {e}")
            # 오류 발생 시 검은색 더미 이미지 반환
            dummy = Image.new('RGB', (224, 224), color='black')
            if self.transform:
                dummy = self.transform(dummy)
            return {
                'path': image_path,
                'image': dummy,
                'error': True
            }
    
    def mask_people(self, image):
        """이미지에서 사람을 감지하고 마스킹"""
        try:
            # DETR 모델을 사용하여 사람 감지
            inputs = self.people_detector(images=image, return_tensors="pt")
            outputs = self.people_detector.model(**inputs)
            
            # DETR 모델 결과 처리
            target_sizes = torch.tensor([image.size[::-1]])
            results = self.people_detector.post_process_object_detection(
                outputs, target_sizes=target_sizes, threshold=0.7
            )[0]
            
            # NumPy 배열로 변환
            img_np = np.array(image)
            
            # 사람으로 감지된 영역 마스킹 (평균 색상 또는 가우시안 블러로 대체)
            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                if label == 1:  # COCO 데이터셋에서 '사람' 클래스는 인덱스 1
                    # 박스 좌표 추출
                    xmin, ymin, xmax, ymax = box.int().tolist()
                    
                    # 영역 마스킹 (다양한 방법 중 하나 선택)
                    # 1. 평균 색상으로 대체
                    # mean_color = np.mean(img_np, axis=(0, 1))
                    # img_np[ymin:ymax, xmin:xmax] = mean_color
                    
                    # 2. 가우시안 블러 적용
                    roi = img_np[ymin:ymax, xmin:xmax]
                    if roi.size > 0:  # 유효한 영역인 경우
                        blurred = cv2.GaussianBlur(roi, (51, 51), 0)
                        img_np[ymin:ymax, xmin:xmax] = blurred
            
            # 이미지로 변환하여 반환
            masked_image = Image.fromarray(img_np)
            return masked_image
        
        except Exception as e:
            print(f"사람 마스킹 중 오류 발생: {e}")
            return image  # 오류 시 원본 이미지 반환

def extract_datetime(image_path):
    """이미지의 EXIF 데이터에서 촬영 시간 추출"""
    try:
        img = Image.open(image_path)
        exif_data = img._getexif()
        if exif_data:
            for tag_id, value in exif_data.items():
                tag = TAGS.get(tag_id, tag_id)
                if tag == 'DateTimeOriginal':
                    return datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
    except Exception as e:
        print(f"Error extracting datetime from {image_path}: {e}")
    return None

def create_people_detector():
    """사람 감지 모델 초기화"""
    try:
        print("사람 감지 모델 로딩 중...")
        processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", use_fast=True)
        model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")
        
        # 감지 모델을 GPU로 이동 (가능한 경우)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to(device)
        model.eval()
        
        # 사용자 정의 감지기 클래스 생성
        class PeopleDetector:
            def __init__(self, processor, model):
                self.processor = processor
                self.model = model
                self.device = device
            
            def __call__(self, images, return_tensors="pt"):
                inputs = self.processor(images=images, return_tensors=return_tensors)
                # 입력을 디바이스로 이동
                inputs = {k: v.to(self.device) if isinstance(v, torch.Tensor) else v 
                          for k, v in inputs.items()}
                return inputs
            
            def post_process_object_detection(self, outputs, **kwargs):
                return self.processor.post_process_object_detection(outputs, **kwargs)
        
        return PeopleDetector(processor, model)
    
    except Exception as e:
        print(f"사람 감지 모델 로드 실패: {e}")
        print("사람 마스킹 비활성화 중...")
        return None

def extract_features_with_dinov2(image_paths, batch_size=8, remove_people=True):
    """DINOv2-giant 모델을 사용하여 이미지 특징 추출 (사람 마스킹 옵션 포함)"""
    print("DINOv2-giant 모델 로딩 중...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"사용 중인 디바이스: {device}")
    
    # 사람 감지 모델 초기화 (사람 제거 옵션이 활성화된 경우)
    people_detector = create_people_detector() if remove_people else None
    
    try:
        # DINOv2 모델 및 프로세서 로드
        processor = AutoImageProcessor.from_pretrained("facebook/dinov2-giant", use_fast=True)
        model = AutoModel.from_pretrained("facebook/dinov2-giant")
        model = model.to(device)
        model.eval()
        
        # 이미지 전처리 변환
        transform = transforms.Compose([
            transforms.Resize((518, 518)),  # DINOv2 권장 사이즈
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        
        # 데이터셋 및 데이터로더 생성 (사람 마스킹 옵션 포함)
        dataset = ImageDataset(
            image_paths, 
            transform=transform, 
            remove_people=remove_people, 
            people_detector=people_detector
        )
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)
        
        features = []
        valid_paths = []
        datetime_list = []
        
        print("이미지 특징 추출 중...")
        with torch.no_grad():
            for batch in tqdm.tqdm(dataloader, desc="특징 추출"):
                images = batch['image'].to(device)
                paths = batch['path']
                errors = batch.get('error', [False] * len(paths))
                
                # 유효한 이미지만 처리
                valid_indices = [i for i, error in enumerate(errors) if not error]
                
                if valid_indices:
                    # DINOv2 모델로 특징 추출
                    outputs = model(images[valid_indices])
                    # [CLS] 토큰의 특징 벡터 사용
                    batch_features = outputs.last_hidden_state[:, 0, :].cpu().numpy()
                    
                    for i, idx in enumerate(valid_indices):
                        features.append(batch_features[i])
                        valid_paths.append(paths[idx])
                        
                        # 시간 정보 추출
                        dt = extract_datetime(paths[idx])
                        datetime_list.append(dt)
        
        print(f"총 {len(valid_paths)}개 이미지에서 특징 추출 완료")
        return features, valid_paths, datetime_list
        
    except Exception as e:
        print(f"DINOv2 모델 사용 중 오류 발생: {e}")
        # 오류 시 기본 특징 추출 방식으로 대체
        print("기본 특징 추출 방식으로 대체합니다...")
        return extract_features_with_opencv(image_paths, remove_people, people_detector)

def extract_features_with_opencv(image_paths, remove_people=True, people_detector=None):
    """OpenCV를 사용한 백업 특징 추출 방식 (사람 마스킹 옵션 포함)"""
    print("OpenCV를 사용하여 특징 추출 중...")
    features_list = []
    valid_paths = []
    datetime_list = []
    
    for i, path in enumerate(image_paths):
        if i % 20 == 0:
            print(f"  {i}/{len(image_paths)} 처리 중...")
        
        try:
            # 이미지 로딩 (PIL로 로드하여 필요시 사람 마스킹 적용)
            if remove_people and people_detector is not None:
                pil_img = Image.open(path).convert('RGB')
                pil_img = ImageDataset.mask_people(pil_img, people_detector)
                img = np.array(pil_img)
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # PIL은 RGB, OpenCV는 BGR
            else:
                # 일반적인 OpenCV 로딩
                img = cv2.imread(path)
            
            if img is None:
                continue
            
            # 이미지 크기 조정
            img = cv2.resize(img, (224, 224))
            
            # 색상 히스토그램 특징 추출
            hist_features = []
            for i in range(3):  # BGR 채널
                hist = cv2.calcHist([img], [i], None, [64], [0, 256])
                hist = cv2.normalize(hist, hist).flatten()
                hist_features.extend(hist)
            
            # 간단한 전역 특징 추가
            for i in range(3):
                hist_features.append(float(np.mean(img[:,:,i])))
                hist_features.append(float(np.std(img[:,:,i])))
            
            features = np.array(hist_features, dtype=float)
            
            # 특징 벡터가 유효한지 확인
            if not np.all(np.isfinite(features)):
                continue
                
            features_list.append(features)
            valid_paths.append(path)
            
            # 시간 정보 추출
            dt = extract_datetime(path)
            datetime_list.append(dt)
            
        except Exception as e:
            print(f"Error processing {path}: {e}")
    
    return features_list, valid_paths, datetime_list

def cluster_images(features, valid_paths, datetime_list, n_clusters=None):
    """이미지 클러스터링"""
    if not features or not valid_paths:
        print("유효한 특징이 추출되지 않았습니다.")
        return None, None
    
    # 특징 데이터 정규화
    print("특징 데이터 정규화 중...")
    X = np.array(features)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_combined = X_scaled
    
    # 시간 특징 추가 (있는 경우)
    # has_time_data = all(dt is not None for dt in datetime_list)
    # if has_time_data:
    #     print("시간 정보를 활용하여 클러스터링합니다...")
    #     # 시간을 타임스탬프로 변환하고 정규화
    #     timestamps = np.array([(dt - datetime(1970, 1, 1)).total_seconds() for dt in datetime_list]).reshape(-1, 1)
    #     time_scaler = StandardScaler()
    #     timestamps_scaled = time_scaler.fit_transform(timestamps)
        
    #     # 가중치 적용 (시간 정보에 가중치 부여)
    #     time_weight = 0.3
    #     feature_weight = 1.0 - time_weight
        
    #     # 특징과 시간 정보 결합
    #     X_combined = np.hstack([X_scaled * feature_weight, timestamps_scaled * time_weight])
    # else:
    #     print("시간 정보가 없어 이미지 특징만으로 클러스터링합니다...")
    #     X_combined = X_scaled
    
    # 클러스터링 알고리즘 선택 및 적용
    print("클러스터링 중...")
    if n_clusters and n_clusters > 0:
        print(f"K-means 클러스터링으로 {n_clusters}개 구역으로 분류합니다.")
        # K-means 클러스터링 
        clusterer = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    else:
        print("DBSCAN 클러스터링으로 자동 구역 분류를 수행합니다.")
        # DBSCAN 클러스터링 (클러스터 수를 자동으로 결정)
        eps = 0.5  # 밀도 기반 클러스터링의 이웃 거리 임계값
        min_samples = 5  # 핵심 포인트 기준 최소 샘플 수
        clusterer = DBSCAN(eps=eps, min_samples=min_samples)
    
    labels = clusterer.fit_predict(X_combined)
    
    return valid_paths, labels

def organize_images_by_cluster(image_paths, labels, output_dir):
    """클러스터링 결과에 따라 이미지 정리"""
    # 클러스터별 디렉토리 생성
    unique_labels = set(labels)
    
    for label in unique_labels:
        # -1은 DBSCAN에서 노이즈를 나타냄
        if label == -1:
            cluster_dir = os.path.join(output_dir, "uncategorized")
        else:
            cluster_dir = os.path.join(output_dir, f"zone_{label+1}")
        
        os.makedirs(cluster_dir, exist_ok=True)
    
    # 각 이미지를 해당 클러스터 디렉토리로 복사
    print("이미지를 구역별 폴더로 복사하는 중...")
    for path, label in zip(image_paths, labels):
        if label == -1:
            dest_dir = os.path.join(output_dir, "uncategorized")
        else:
            dest_dir = os.path.join(output_dir, f"zone_{label+1}")
        
        # 원본 파일명 유지하면서 복사
        filename = os.path.basename(path)
        shutil.copy2(path, os.path.join(dest_dir, filename))
    
    # 각 클러스터에 몇 개의 이미지가 있는지 출력
    print("\n분류 결과:")
    for label in unique_labels:
        if label == -1:
            dir_name = "uncategorized"
        else:
            dir_name = f"zone_{label+1}"
        
        cluster_dir = os.path.join(output_dir, dir_name)
        num_images = len(os.listdir(cluster_dir))
        print(f"{dir_name}: {num_images}개 이미지")

def visualize_clusters(image_paths, labels, output_dir):
    """클러스터링 결과 시각화"""
    # 각 클러스터에서 3개의 샘플 이미지를 보여줌
    unique_labels = sorted(set(labels))
    if -1 in unique_labels:  # uncategorized를 마지막에 표시
        unique_labels.remove(-1)
        unique_labels.append(-1)
    
    # 시각화를 위한 그리드 설정
    n_clusters = len(unique_labels)
    
    # 클러스터가 없으면 시각화를 건너뜁니다
    if n_clusters == 0:
        print("시각화할 클러스터가 없습니다.")
        return
    
    # 각 클러스터마다 최대 3개의 이미지를 표시
    max_samples = 3
    fig, axes = plt.subplots(n_clusters, max_samples, figsize=(15, 5 * n_clusters))
    
    # 단일 클러스터이거나 단일 샘플인 경우 축 처리
    if n_clusters == 1:
        axes = np.array([axes])  # 2D 배열로 만들기
    
    print("클러스터 시각화 생성 중...")
    for i, label in enumerate(unique_labels):
        # 해당 클러스터의 이미지 경로 가져오기
        cluster_images = [path for path, lbl in zip(image_paths, labels) if lbl == label]
        
        # 최대 3개의 샘플 선택
        samples = cluster_images[:max_samples]
        
        # 샘플 이미지 표시
        for j in range(max_samples):
            ax = axes[i, j] if n_clusters > 1 else axes[j]
            
            if j < len(samples):  # 샘플이 있는 경우
                try:
                    sample = samples[j]
                    img = cv2.imread(sample)
                    if img is not None:
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR, matplotlib은 RGB
                        ax.imshow(img)
                        ax.set_title(f"Cluster {label+1 if label != -1 else 'Uncategorized'}\nSample {j+1}")
                    else:
                        ax.text(0.5, 0.5, "이미지 로드 실패", ha='center', va='center')
                except Exception as e:
                    print(f"Error displaying sample {samples[j] if j < len(samples) else 'unknown'}: {e}")
                    ax.text(0.5, 0.5, "이미지 표시 오류", ha='center', va='center')
            else:  # 샘플이 부족한 경우
                ax.text(0.5, 0.5, "샘플 없음", ha='center', va='center')
            
            ax.axis('off')
    
    plt.tight_layout()
    try:
        vis_path = os.path.join(output_dir, "cluster_samples.png")
        plt.savefig(vis_path)
        plt.close()
        print(f"클러스터 시각화가 {vis_path}에 저장되었습니다.")
    except Exception as e:
        print(f"시각화 저장 중 오류 발생: {e}")
        plt.close()

def save_processed_examples(original_paths, output_dir, people_detector, count=5):
    """사람이 마스킹된 이미지 예시 저장"""
    if people_detector is None:
        return
    
    print("사람 마스킹 처리 예시 생성 중...")
    example_dir = os.path.join(output_dir, "people_masking_examples")
    os.makedirs(example_dir, exist_ok=True)
    
    # 무작위로 몇 개의 이미지 선택
    import random
    sample_paths = random.sample(original_paths, min(count, len(original_paths)))
    
    for i, path in enumerate(sample_paths):
        try:
            # 원본 이미지 로드
            original_img = Image.open(path).convert('RGB')
            
            # 사람 마스킹 적용
            masked_img = ImageDataset.mask_people(ImageDataset(), original_img, people_detector)
            
            # 원본과 마스킹된 이미지 나란히 저장
            fig, axes = plt.subplots(1, 2, figsize=(12, 6))
            
            # 원본 이미지
            axes[0].imshow(np.array(original_img))
            axes[0].set_title("원본 이미지")
            axes[0].axis('off')
            
            # 마스킹된 이미지
            axes[1].imshow(np.array(masked_img))
            axes[1].set_title("사람 마스킹 처리된 이미지")
            axes[1].axis('off')
            
            plt.tight_layout()
            example_path = os.path.join(example_dir, f"example_{i+1}.png")
            plt.savefig(example_path)
            plt.close()
            
        except Exception as e:
            print(f"예시 이미지 생성 중 오류: {e}")
    
    print(f"사람 마스킹 처리 예시가 {example_dir}에 저장되었습니다.")

def main():
    print("="*50)
    print("잔디 깎기 작업 사진 구역별 분류 프로그램 (사람 특징 무시 버전)")
    print("="*50)
    
    # 디렉토리 설정 확인
    print(f"입력 디렉토리: {IMAGE_DIR}")
    print(f"출력 디렉토리: {OUTPUT_DIR}")
    
    # 출력 디렉토리 생성
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # 지원하는 이미지 확장자
    valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif'}
    
    # 모든 이미지 파일 경로 수집
    print("이미지 파일 검색 중...")
    image_paths = []
    for root, _, files in os.walk(IMAGE_DIR):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            if ext in valid_extensions:
                image_paths.append(os.path.join(root, file))
    
    print(f"총 {len(image_paths)}개의 이미지를 발견했습니다.")
    
    # 구역 수 입력 (자동 결정을 원하면 0 입력)
    try:
        n_zones = int(input("구역 수를 입력하세요 (자동 결정은 0 입력): "))
    except ValueError:
        print("유효하지 않은 입력입니다. 자동으로 구역을 결정합니다.")
        n_zones = 0
    
    # 사람 특징을 무시하는 옵션 활성화
    remove_people = True
    print(f"사람 특징 무시 옵션: {'활성화' if remove_people else '비활성화'}")
    
    try:
        # 특징 추출
        features, valid_paths, datetime_list = extract_features_with_dinov2(
            image_paths, 
            remove_people=remove_people
        )
        
        # 클러스터링
        valid_paths, labels = cluster_images(
            features, 
            valid_paths, 
            datetime_list, 
            n_clusters=n_zones if n_zones > 0 else None
        )
        
        if valid_paths is None or labels is None:
            print("클러스터링에 실패했습니다.")
            return
        
        # 클러스터링 결과에 따라 이미지 정리
        organize_images_by_cluster(valid_paths, labels, OUTPUT_DIR)
        
        # 클러스터 시각화
        visualize_clusters(valid_paths, labels, OUTPUT_DIR)
        
        # 마스킹 예시 생성 (선택적)
        if remove_people:
            people_detector = create_people_detector()
            if people_detector:
                save_processed_examples(image_paths[:10], OUTPUT_DIR, people_detector)
        
    except Exception as e:
        print(f"처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return
    
    print("\n"+"="*50)
    print(f"이미지 분류가 완료되었습니다.")
    print(f"결과는 {OUTPUT_DIR} 디렉토리에 저장되었습니다.")
    print("="*50)

# if __name__ == "__main__":
#     main()

/Users/aohus/.pyenv/versions/local-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

print("잔디 깎기 작업 사진 구역별 분류 프로그램 (DINOv2 버전)")

# 고정된 입력 및 출력 디렉토리 설정
IMAGE_DIR = "./1차"
OUTPUT_DIR = IMAGE_DIR + "_claude2"


# 디렉토리 설정 확인
print(f"입력 디렉토리: {IMAGE_DIR}")
print(f"출력 디렉토리: {OUTPUT_DIR}")

# 출력 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 지원하는 이미지 확장자
valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif'}

# 모든 이미지 파일 경로 수집
print("이미지 파일 검색 중...")
image_paths = []
for root, _, files in os.walk(IMAGE_DIR):
    for file in files:
        ext = os.path.splitext(file)[1].lower()
        if ext in valid_extensions:
            image_paths.append(os.path.join(root, file))

print(f"총 {len(image_paths)}개의 이미지를 발견했습니다.")


잔디 깎기 작업 사진 구역별 분류 프로그램 (DINOv2 버전)
입력 디렉토리: ./1차
출력 디렉토리: ./1차_claude2
이미지 파일 검색 중...
총 232개의 이미지를 발견했습니다.


In [3]:
remove_people = False
print(f"사람 특징 무시 옵션: {'활성화' if remove_people else '비활성화'}")

# DINOv2 모델을 사용하여 이미지 특징 추출
features, valid_paths, datetime_list = extract_features_with_dinov2(
    image_paths, 
    remove_people=remove_people
)

사람 특징 무시 옵션: 비활성화
DINOv2-giant 모델 로딩 중...
사용 중인 디바이스: cpu


`use_fast` is set to `True` but the image processor class does not have a fast version.  Falling back to the slow version.


이미지 특징 추출 중...


특징 추출: 100%|██████████| 29/29 [14:29<00:00, 29.97s/it]

총 232개 이미지에서 특징 추출 완료


In [4]:
# 클러스터링
n_zones = 55
valid_paths, labels = cluster_images(
    features, 
    valid_paths, 
    datetime_list, 
    n_clusters=n_zones if n_zones > 0 else None
)
organize_images_by_cluster(valid_paths, labels, OUTPUT_DIR)

특징 데이터 정규화 중...
클러스터링 중...
K-means 클러스터링으로 55개 구역으로 분류합니다.
이미지를 구역별 폴더로 복사하는 중...

분류 결과:
zone_1: 2개 이미지
zone_2: 6개 이미지
zone_3: 12개 이미지
zone_4: 5개 이미지
zone_5: 5개 이미지
zone_6: 12개 이미지
zone_7: 7개 이미지
zone_8: 6개 이미지
zone_9: 6개 이미지
zone_10: 5개 이미지
zone_11: 6개 이미지
zone_12: 6개 이미지
zone_13: 3개 이미지
zone_14: 4개 이미지
zone_15: 2개 이미지
zone_16: 3개 이미지
zone_17: 4개 이미지
zone_18: 5개 이미지
zone_19: 7개 이미지
zone_20: 5개 이미지
zone_21: 3개 이미지
zone_22: 3개 이미지
zone_23: 3개 이미지
zone_24: 4개 이미지
zone_25: 3개 이미지
zone_26: 3개 이미지
zone_27: 4개 이미지
zone_28: 3개 이미지
zone_29: 6개 이미지
zone_30: 2개 이미지
zone_31: 2개 이미지
zone_32: 2개 이미지
zone_33: 2개 이미지
zone_34: 7개 이미지
zone_35: 5개 이미지
zone_36: 5개 이미지
zone_37: 5개 이미지
zone_38: 2개 이미지
zone_39: 2개 이미지
zone_40: 5개 이미지
zone_41: 4개 이미지
zone_42: 8개 이미지
zone_43: 7개 이미지
zone_44: 2개 이미지
zone_45: 3개 이미지
zone_46: 4개 이미지
zone_47: 1개 이미지
zone_48: 2개 이미지
zone_49: 2개 이미지
zone_50: 3개 이미지
zone_51: 3개 이미지
zone_52: 6개 이미지
zone_53: 2개 이미지
zone_54: 1개 이미지
zone_55: 2개 이미지


In [5]:
# 클러스터 시각화
visualize_clusters(valid_paths, labels, OUTPUT_DIR)

if remove_people:
    people_detector = create_people_detector()c
    if people_detector:
        save_processed_examples(image_paths[:10], OUTPUT_DIR, people_detector)
        
print(f"이미지 분류가 완료되었습니다.")
print(f"결과는 {OUTPUT_DIR} 디렉토리에 저장되었습니다.")

클러스터 시각화 생성 중...


/var/folders/fv/fgltl3gj7gzf7yxy510rkjv00000gn/T/ipykernel_14645/141794884.py:427: UserWarning: Glyph 49368 (\N{HANGUL SYLLABLE SAEM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/var/folders/fv/fgltl3gj7gzf7yxy510rkjv00000gn/T/ipykernel_14645/141794884.py:427: UserWarning: Glyph 54540 (\N{HANGUL SYLLABLE PEUL}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/var/folders/fv/fgltl3gj7gzf7yxy510rkjv00000gn/T/ipykernel_14645/141794884.py:427: UserWarning: Glyph 50630 (\N{HANGUL SYLLABLE EOBS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/var/folders/fv/fgltl3gj7gzf7yxy510rkjv00000gn/T/ipykernel_14645/141794884.py:427: UserWarning: Glyph 51020 (\N{HANGUL SYLLABLE EUM}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/var/folders/fv/fgltl3gj7gzf7yxy510rkjv00000gn/T/ipykernel_14645/141794884.py:430: UserWarning: Glyph 49368 (\N{HANGUL SYLLABLE SAEM}) missing from font(s) DejaVu Sans.
  plt.savefig(vis_path)
/var/folders/fv/fgltl3gj7gzf7yxy510rkjv00000gn/T

클러스터 시각화가 ./1차_claude2/cluster_samples.png에 저장되었습니다.
이미지 분류가 완료되었습니다.
결과는 ./1차_claude2 디렉토리에 저장되었습니다.
